In [0]:
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from keras import layers, Input, models, optimizers
from keras.models import Model, Sequential
from keras.layers import MaxPooling2D, Conv2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

### pattern 별로 이미지 선별했기 때문에 따로 불러와서 분류 작업 불필요

## 얼굴있는 사진 제거 및 새로 저장

In [0]:
face_cascade = cv2.CascadeClassifier(
    './opencv-master/data/haarcascades/haarcascade_frontalface_default.xml')

In [0]:
# 얼굴 분류 함수

def count_face(x):
    face_cascade = cv2.CascadeClassifier('./opencv-master/data/haarcascades/haarcascade_frontalface_default.xml')
    grayImage = cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(grayImage, 1.03, 5)
    return faces.shape[0]

In [0]:
# 폴더 생성 함수

def createfolder(foler_dir):
    try:
        if not os.path.exists(folder_dir):
            os.makedirs(folder_dir)
    except OSError:
        print('Error Creating directory'+folder_dir) 

In [0]:
# noface 하위 폴더 만들기

path = 'C:/Jupyter_Script/img/pattern/' # 가져오는 이미지 폴더 path
file_list = os.listdir(path)

for folder_name in file_list:
    path = 'C:/Jupyter_Script/img/noface_pattern/' # 새로운 이미지 폴더 path
    folder_dir = path + folder_name
    createfolder(folder_dir)

In [0]:
# noface 사진 다시 저장

path = 'C:/Jupyter_Script/img/pattern/'  # 가져오는 이미지 폴더 path
file_list = os.listdir(path)

for _ in range(len(file_list)):
    folder_name = file_list[_]
    new_dir = 'C:/Jupyter_Script/img/noface_pattern/' + folder_name # 저장할 path
    jpg_path = path + file_list[_] + '/'
    jpg_file =  os.listdir(jpg_path)
    a = 0
    for i in range(len(jpg_file)):
        image_name = jpg_path + jpg_file[i]
        img = cv2.imread(image_name)

        try:
            count_face(img)
            
        except:
            new_name = new_dir + '/' + jpg_file[i]
            cv2.imwrite(new_name, img)
    
    if _ % 50 == 0:
        print(_)


## pattern 분류 모델 생성

In [0]:
# generator = tf.keras.preprocessing.image.ImageDataGenerator(
#     rotation_range = 20,
#     width_shift_range = 0.2,
#     height_shift_range = 0.2,
#     rescale = 1. / 255)

In [0]:
model = Sequential()
model.add(Conv2D(16, (3, 3), padding='same', use_bias=False, input_shape=(150,150, 3)))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))
 
model.add(Conv2D(32, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))
 
model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(Flatten())
model.add(Dropout(0.2))
 
model.add(Dense(512, activation='relu'))
model.add(Dense(8, activation='softmax'))                                   # label, calss 갯수
model.summary()
 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 150, 150, 16)      432       
_________________________________________________________________
batch_normalization_9 (Batch (None, 150, 150, 16)      48        
_________________________________________________________________
activation_9 (Activation)    (None, 150, 150, 16)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 38, 38, 16)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 38, 38, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 38, 38, 32)        4608      
_________________________________________________________________
batch_normalization_10 (Batc (None, 38, 38, 32)        96        
__________

In [0]:
# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


# 데이터 셋
train_datagen = ImageDataGenerator(rescale=1./255,
#                                    rotation_range=10,
#                                    width_shift_range=0.2,
#                                    height_shift_range=0.2,
#                                    shear_range=0.7,
#                                    zoom_range=[0.9, 2.2],
#                                    horizontal_flip=True,
#                                    vertical_flip=True,
#                                    fill_mode='nearest',
                                   validation_split=0.33)
 
training_set = train_datagen.flow_from_directory('C:/Jupyter_Script/img/pattern',
                                                 target_size = (150,150),
                                                 batch_size = 15,
                                                 class_mode = 'categorical',
                                                 subset="training"
                                                )
validation_set = train_datagen.flow_from_directory('C:/Jupyter_Script/img/pattern',
                                                 target_size = (150,150),
                                                 batch_size = 10,
                                                 class_mode = 'categorical',
                                                 subset="validation"
                                                  )
 

Found 4681 images belonging to 8 classes.
Found 2305 images belonging to 8 classes.


In [0]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('./log.csv', append=True, separator=';')
 
 
hist = model.fit_generator(training_set,
                         steps_per_epoch = 10, #20
                         epochs = 50,  #1000
                         validation_data = validation_set,
                         validation_steps = 10,
                         callbacks=[csv_logger])
 
 

Epoch 1/50
10/10 [==============================] - 37s 4s/step - loss: 2.5980 - acc: 0.2000 - val_loss: 2.2641 - val_acc: 0.2500
Epoch 2/50
10/10 [==============================] - 38s 4s/step - loss: 2.2729 - acc: 0.2400 - val_loss: 2.1885 - val_acc: 0.2500
Epoch 3/50
10/10 [==============================] - 37s 4s/step - loss: 2.1498 - acc: 0.2333 - val_loss: 2.1410 - val_acc: 0.3100
Epoch 4/50
10/10 [==============================] - 40s 4s/step - loss: 1.9496 - acc: 0.3267 - val_loss: 2.0283 - val_acc: 0.2600
Epoch 5/50
10/10 [==============================] - 39s 4s/step - loss: 2.0116 - acc: 0.2000 - val_loss: 2.1069 - val_acc: 0.2000
Epoch 6/50
10/10 [==============================] - 37s 4s/step - loss: 2.0584 - acc: 0.2533 - val_loss: 1.9549 - val_acc: 0.2700
Epoch 7/50
10/10 [==============================] - 48s 5s/step - loss: 1.9600 - acc: 0.2267 - val_loss: 1.9122 - val_acc: 0.2300
Epoch 8/50
10/10 [==============================] - 46s 5s/step - loss: 2.0190 - acc: 0.25

In [0]:
# 모델 저장


from keras.models import load_model, model_from_json
model.save('model_pattern.h5')                       # 모델(architecture, weight 동시 저장) 저장 이름


# 모델 아키텍처를 json 형, 식으로 저장                 # 모델 architecture 만 저장 
#json_string = model.to_json()                       # json 파일에서 모델 아키텍처 재구성
#model = model_from_json(json_string) 


# 모델 Weights 저장
#model.save_weights()


ModuleNotFoundError: No module named 'models'

In [0]:
from keras.models import load_model
model = load_model('model_pattern.h5')

In [0]:
# output = classifier.predict_generator(test_set, steps=5)
# print(test_set.class_indices)
# print(output)
 
# 모델 평가하기
print("-- Evaluate --")
 
scores = model.evaluate_generator(
            validation_set,
            steps = 10)
 
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
 
# 모델 예측하기
print("-- Predict --")
 
output = model.predict_generator(
            validation_set,
            steps = 100)
print(validation_set.class_indices)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
 

-- Evaluate --
acc: 31.00%
-- Predict --
{'camo': 0, 'check': 1, 'dot': 2, 'floral': 3, 'graphic': 4, 'lace': 5, 'leopard': 6, 'stripe': 7}


In [0]:
# 훈련 세트의 정확도와 테스트 세트의 정확도 사이의 차이는 과대적합(overfitting) 때문
# 과대적합은 머신러닝 모델이 훈련 데이터보다 새로운 데이터에서 성능이 낮아지는 현상

In [0]:
# 모델 예측
# predict는 확률을, predict_classes는 label 값을 알려줌

In [0]:
# 학습 과정 정확도, 손실함수 변화 그래프로 보기


import matplotlib.pyplot as plt
 
fig, loss_ax = plt.subplots()
 
acc_ax = loss_ax.twinx()
 
loss_ax.plot(hist.history['loss'], 'y', label='train loss')
#loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
##loss_ax.set_ylim([0.0, 0.5])
 
acc_ax.plot(hist.history['acc'], 'b', label='train acc')
#acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
##acc_ax.set_ylim([0.8, 1.0])
 
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')
 
loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')
 
plt.show()

In [0]:
# 모델 architecture 보기


from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

%matplotlib inline

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [0]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint


# moniter: 대상 값, mode: min/max, verbose=1 설정시 언제 멈췄는지 출력
# patience: 성능없는 에폭 허용 범위, baseline: 값 설정

#es = EarlyStopping(monitor='val_loss', mode='min')
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)



In [0]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
mc = ModelCheckpoint('best_model_pattern.h5', monitor='val_loss', mode='min', save_best_only=True)

In [0]:
hist = model.fit_generator(training_set,
                         steps_per_epoch = 20, #20
                         epochs = 50,  #1000
                         validation_data = validation_set,
                         validation_steps = 10,
                         callbacks=[es,mc])
 

Epoch 1/50
20/20 [==============================] - 59s 3s/step - loss: 1.3556 - acc: 0.5167 - val_loss: 1.5213 - val_acc: 0.4500
Epoch 2/50
20/20 [==============================] - 56s 3s/step - loss: 1.4636 - acc: 0.4500 - val_loss: 1.6483 - val_acc: 0.3900
Epoch 3/50
20/20 [==============================] - 57s 3s/step - loss: 1.3835 - acc: 0.4533 - val_loss: 1.7704 - val_acc: 0.3800
Epoch 4/50
20/20 [==============================] - 57s 3s/step - loss: 1.4579 - acc: 0.4667 - val_loss: 1.8449 - val_acc: 0.3200
Epoch 5/50
20/20 [==============================] - 59s 3s/step - loss: 1.3929 - acc: 0.5200 - val_loss: 1.6442 - val_acc: 0.3600
Epoch 6/50
20/20 [==============================] - 56s 3s/step - loss: 1.3999 - acc: 0.4667 - val_loss: 1.7846 - val_acc: 0.3200
Epoch 7/50
20/20 [==============================] - 56s 3s/step - loss: 1.4493 - acc: 0.4900 - val_loss: 2.0274 - val_acc: 0.3000
Epoch 8/50
20/20 [==============================] - 56s 3s/step - loss: 1.3620 - acc: 0.52